## Preprocessing

### 0. 사용 라이브러리

In [5]:
import pandas as pd
import numpy as np

from konlpy.tag import Okt

# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install git+https://github.com/ssut/py-hanspell.git

from pykospacing import Spacing
from hanspell import spell_checker

### 1. 데이터 불러오기

In [5]:
# 데이터 불러오기

train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')


In [6]:
train_data.info() # 150000 row

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
test_data.info() # 50000 row

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


### 2. 데이터 정제

#### 2.1 중복값 처리

In [8]:
# train_data 중복값 확인

print('id 중복값: ', train_data['id'].notnull().sum() - len(train_data['id'].unique()))
print('document 중복값: ', train_data['document'].notnull().sum() - len(train_data['document'].unique()))

print('label 값: ', train_data['label'].unique()) # 0, 1

id 중복값:  0
document 중복값:  3812
label 값:  [0 1]


In [9]:
# test_data 중복값 확인

print('id 중복값: ', test_data['id'].notnull().sum() - len(test_data['id'].unique()))
print('document 중복값: ', test_data['document'].notnull().sum() - len(test_data['document'].unique()))

print('label 값: ', test_data['label'].unique()) # 0, 1

id 중복값:  0
document 중복값:  839
label 값:  [1 0]


In [10]:
# 중복값 제거

train_data.drop_duplicates(['document'], inplace=True)
test_data.drop_duplicates(['document'], inplace=True)

print(len(train_data), len(test_data))

146183 49158


#### 2.2 구두점, 특수문자 제거

In [11]:
train_data['document'] = train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','') # [ㄱ - ㅎ, ㅏ - ㅣ, 가 - 핳] 제외 제거
train_data['document'] = train_data['document'].str.replace('^ +', '') # 화이트 스페이스 제거

test_data['document'] = test_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_data['document'] = test_data['document'].str.replace('^ +', '')

C:\Users\User\AppData\Local\Temp/ipykernel_13008/2848603368.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','') # [ㄱ - ㅎ, ㅏ - ㅣ, 가 - 핳] 제외 제거
C:\Users\User\AppData\Local\Temp/ipykernel_13008/2848603368.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', '') # 화이트 스페이스 제거
C:\Users\User\AppData\Local\Temp/ipykernel_13008/2848603368.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
C:\Users\User\AppData\Local\Temp/ipykernel_13008/2848603368.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', '')


#### 2.3 결측치 처리

In [12]:
# 결측치 확인

print(len(train_data[train_data['document']==""]), train_data['document'].isnull().sum())
train_data.replace("", np.nan, inplace=True) # 비어있는 값 null로 변환
train_data[train_data['document'].isnull()==True]

789 1


,id,document,label
404,4221289,NaN,0
412,9509970,NaN,1
470,10147571,NaN,1
584,7117896,NaN,0
593,6478189,NaN,0
...,...,...,...
149309,6715725,NaN,1
149358,6780491,NaN,0
149364,8014701,NaN,1
149630,3508604,NaN,0


In [13]:
# 결측치 확인

print(len(test_data[test_data['document']==""]), test_data['document'].isnull().sum())
test_data.replace("", np.nan, inplace=True) # 비어있는 값 null로 변환
test_data[test_data['document'].isnull()==True]

305 1


,id,document,label
1,9274899,NaN,0
116,6910486,NaN,1
254,4976468,NaN,0
468,7517102,NaN,0
504,2096424,NaN,0
...,...,...,...
49420,5187147,NaN,0
49459,6381245,NaN,1
49803,5309713,NaN,1
49871,9767991,NaN,0


In [14]:
# 결측치 제거

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [15]:
# 결측치 확인

print(train_data['document'].isnull().sum(), test_data['document'].isnull().sum())
print(len(train_data), len(test_data))

0 0
145393 48852


### 3. 띄어쓰기, 맞춤법

In [16]:
spacing = Spacing()

test_sentence = train_data.iloc[1, 1]
spaced_sentence = spacing(test_sentence)
print(test_sentence)
print(spaced_sentence)

흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
흠포스터 보고 초딩 영화 줄오버 연기조차 가볍지 않구나


In [17]:
test_sentence = train_data.iloc[2, 1]
spaced_sentence = spacing(test_sentence)
print(test_sentence)
print(spaced_sentence)

너무재밓었다그래서보는것을추천한다
너무 재밓었다 그래서 보는 것을 추천한다


In [18]:
test_sentence = train_data.iloc[10, 1]
spaced_sentence = spacing(test_sentence)
print(test_sentence)
print(spaced_sentence)

걍인피니트가짱이다진짜짱이다
걍 인피니트가 짱이 다 진짜 짱이다


In [ ]:
# train_data['document'] = train_data['document'].apply(spacing)


In [24]:
# spaced_train_data = train_data.copy()
# spaced_train_data.to_csv('ratings_train_spaced.csv', index=False)

# csv 파일 생성 이후

In [23]:
train_data = pd.read_csv('ratings_train_spaced.csv')

In [24]:
spaced_train_data = train_data.copy()
train_data.iloc[10]

id                      9008700
document    걍 인피니트가 짱이 다 진짜 짱이다
label                         1
Name: 10, dtype: object

### 3. 토큰화

형태소 분석기 종류
Okt(Open Korea Text), 메캅(Mecab), 코모란(Komoran), 한나눔(Hannanum), 꼬꼬마(Kkma)

#### 3.1 Okt(Open Korea Text)

In [8]:
test_sentence = train_data.iloc[10, 1]

In [27]:
from konlpy.tag import Okt

okt = Okt()

print('OKT 형태소 분석 :',okt.morphs(test_sentence))
print('OKT 품사 태깅 :',okt.pos(test_sentence))
print('OKT 명사 추출 :',okt.nouns(test_sentence)) 

OKT 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이다']
OKT 품사 태깅 : [('걍', 'Adverb'), ('인피니트', 'Noun'), ('가', 'Josa'), ('짱', 'Noun'), ('이', 'Josa'), ('다', 'Adverb'), ('진짜', 'Noun'), ('짱', 'Noun'), ('이다', 'Josa')]
OKT 명사 추출 : ['인피니트', '짱', '진짜', '짱']


#### 3.2 꼬꼬마 Kkma

In [10]:
from konlpy.tag import Kkma

kkma = Kkma()

print('꼬꼬마 형태소 분석 :',kkma.morphs(test_sentence))
print('꼬꼬마 품사 태깅 :',kkma.pos(test_sentence))
print('꼬꼬마 명사 추출 :',kkma.nouns(test_sentence))  

꼬꼬마 형태소 분석 : ['걍', '인피', '니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
꼬꼬마 품사 태깅 : [('걍', 'MAG'), ('인피', 'NNG'), ('니트', 'NNG'), ('가', 'JKS'), ('짱', 'NNG'), ('이', 'JKS'), ('다', 'MAG'), ('진짜', 'MAG'), ('짱', 'NNG'), ('이', 'VCP'), ('다', 'EFN')]
꼬꼬마 명사 추출 : ['인피', '인피니트', '니트', '짱']


#### 3.3 은전한닢 형태소 분석기 mecab

https://eunjeon.blogspot.com/

https://cleancode-ws.tistory.com/97

https://m.blog.naver.com/PostView.nhn?blogId=aul-_-&logNo=221557243190&proxyReferer=https:%2F%2Fwww.google.com%2F

In [8]:
!pip install eunjeon

  Using cached eunjeon-0.4.0.tar.gz (34.7 MB)
  Created wheel for eunjeon: filename=eunjeon-0.4.0-cp39-cp39-win_amd64.whl size=35021862 sha256=a08fc2c709eccf4b9fdfc6ce3af8543969c3b90a15d317ef01efc749fbe8de2f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\51\05\fb\48ff3bf6804f7cea4a3f7be6300a5b19618007c35d2064a753
Successfully built eunjeon


In [9]:
from eunjeon import Mecab
m=Mecab()

print('은전한닢 형태소 분석 :',m.morphs(test_sentence))
print('은전한닢 품사 태깅 :',m.pos(test_sentence))
print('은전한닢 명사 추출 :',m.nouns(test_sentence))  

은전한닢 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
은전한닢 품사 태깅 : [('걍', 'UNKNOWN'), ('인피니트', 'NNP'), ('가', 'JKS'), ('짱', 'NNG'), ('이', 'JKS'), ('다', 'MAG'), ('진짜', 'MAG'), ('짱', 'NNG'), ('이', 'VCP'), ('다', 'EC')]
은전한닢 명사 추출 : ['인피니트', '짱', '짱']


#### 3.4 코모란(Komoran)

In [15]:
# Komoran(코모란)
from konlpy.tag import Komoran
komoran=Komoran()  

print('코모란 형태소 분석 :',komoran.morphs(test_sentence))
print('코모란 품사 태깅 :',komoran.pos(test_sentence))  
print('코모란 명사 추출 :',komoran.nouns(test_sentence))  

코모란 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
코모란 품사 태깅 : [('걍', 'NA'), ('인피니트', 'NNP'), ('가', 'JKS'), ('짱', 'MAG'), ('이', 'JKS'), ('다', 'MAG'), ('진짜', 'MAG'), ('짱', 'MAG'), ('이', 'VCP'), ('다', 'EC')]
코모란 명사 추출 : ['인피니트']


#### 3.5 한나눔(Hannanum)

In [16]:
# Hannanum(한나눔)
from konlpy.tag import Hannanum
tokenizer=Hannanum()  
print('한나눔 형태소 분석 :',tokenizer.morphs(test_sentence))
print('한나눔 품사 태깅 :',tokenizer.pos(test_sentence))  
print('한나눔 명사 추출 :',tokenizer.nouns(test_sentence))  

한나눔 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
한나눔 품사 태깅 : [('걍', 'N'), ('인피니트', 'N'), ('가', 'J'), ('짱', 'N'), ('이', 'J'), ('다', 'M'), ('진짜', 'N'), ('짱', 'N'), ('이', 'J'), ('다', 'E')]
한나눔 명사 추출 : ['걍', '인피니트', '짱', '진짜', '짱']


### 불용어

In [29]:
# 불용어 사전 불러오기
stopwords = pd.read_csv("stopwords.csv", encoding='CP949')
stopwords = list(stopwords['stopwords'])

In [30]:
from nltk.corpus.reader.wordlist import WordListCorpusReader 
stopwords = WordListCorpusReader('.', ['stopwords.csv'], encoding='CP949') 
print(stopwords.words()) 


['stopwords', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자

In [67]:
print(stopwords.fileids())

['stopwords.csv']


In [19]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
from tqdm import tqdm

#### 3.3 은전한닢 형태소 분석기 mecab 사용 토큰화

In [11]:
from eunjeon import Mecab
m=Mecab()

print('은전한닢 형태소 분석 :',m.morphs(test_sentence))
print('은전한닢 품사 태깅 :',m.pos(test_sentence))
print('은전한닢 명사 추출 :',m.nouns(test_sentence))  

은전한닢 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
은전한닢 품사 태깅 : [('걍', 'UNKNOWN'), ('인피니트', 'NNP'), ('가', 'JKS'), ('짱', 'NNG'), ('이', 'JKS'), ('다', 'MAG'), ('진짜', 'MAG'), ('짱', 'NNG'), ('이', 'VCP'), ('다', 'EC')]
은전한닢 명사 추출 : ['인피니트', '짱', '짱']


In [34]:
mecab_x_train = []
for sentence in tqdm(train_data['document']): # 진행상황 Progress Bar를 위한 tqdm library
    tokenized_sentence = m.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords.words()] # 불용어 제거
    mecab_x_train.append(stopwords_removed_sentence)

100%|██████████| 145393/145393 [21:55<00:00, 110.53it/s]


In [35]:
pd.Series(mecab_x_train).to_csv("mecab_x_train.csv")

#### 3.4 코모란 KOMORAN 사용 토큰화

In [36]:
# Komoran(코모란)
from konlpy.tag import Komoran
komoran=Komoran()  

print('코모란 형태소 분석 :',komoran.morphs(test_sentence))
print('코모란 품사 태깅 :',komoran.pos(test_sentence))  
print('코모란 명사 추출 :',komoran.nouns(test_sentence))  

코모란 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
코모란 품사 태깅 : [('걍', 'NA'), ('인피니트', 'NNP'), ('가', 'JKS'), ('짱', 'MAG'), ('이', 'JKS'), ('다', 'MAG'), ('진짜', 'MAG'), ('짱', 'MAG'), ('이', 'VCP'), ('다', 'EC')]
코모란 명사 추출 : ['인피니트']


In [38]:
komoran_x_train = []
for sentence in tqdm(train_data['document']): # 진행상황 Progress Bar를 위한 tqdm library
    tokenized_sentence = komoran.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords.words()] # 불용어 제거
    komoran_x_train.append(stopwords_removed_sentence)

100%|██████████| 145393/145393 [23:10<00:00, 104.59it/s]


In [39]:
pd.Series(komoran_x_train).to_csv("komoran_x_train.csv")

#### 3.5 한나눔 Hannanum 사용 토큰화

In [40]:
# Hannanum(한나눔)
from konlpy.tag import Hannanum
hannanum=Hannanum()  
print('한나눔 형태소 분석 :',hannanum.morphs(test_sentence))
print('한나눔 품사 태깅 :',hannanum.pos(test_sentence))  
print('한나눔 명사 추출 :',hannanum.nouns(test_sentence))  

한나눔 형태소 분석 : ['걍', '인피니트', '가', '짱', '이', '다', '진짜', '짱', '이', '다']
한나눔 품사 태깅 : [('걍', 'N'), ('인피니트', 'N'), ('가', 'J'), ('짱', 'N'), ('이', 'J'), ('다', 'M'), ('진짜', 'N'), ('짱', 'N'), ('이', 'J'), ('다', 'E')]
한나눔 명사 추출 : ['걍', '인피니트', '짱', '진짜', '짱']


In [42]:
hannanum_x_train = []
for sentence in tqdm(train_data['document']): # 진행상황 Progress Bar를 위한 tqdm library
    tokenized_sentence = hannanum.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords.words()] # 불용어 제거
    hannanum_x_train.append(stopwords_removed_sentence)

100%|██████████| 145393/145393 [30:19<00:00, 79.92it/s] 


In [43]:
pd.Series(hannanum_x_train).to_csv("hannanum_x_train.csv")